# ETL (Extract Transform Load)
## Extraer, Transformar, Carga del archivo Reviews
Es un proceso que se utiliza para mover datos de una fuente (Extract), realizar modificaciones en esos datos según sea necesario (Transform), y cargar los datos resultantes en un destino deseado (Load).

##### Convertimos el arvhivo JSON a una dataframe con pandas.

In [2]:
# importamos liberias
import pandas as pd
import numpy as np
import ast

def load_json_lines(file_path):
    data = []
    with open(file_path, "r", encoding="utf-8") as file:
        for line in file:
            data.append(ast.literal_eval(line))
    return pd.DataFrame(data)

# Cargar archivos
df_reviews = load_json_lines("australian_user_reviews.json ")

##### Data General

In [3]:
# Vista general del Dataset
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25799 non-null  object
 1   user_url  25799 non-null  object
 2   reviews   25799 non-null  object
dtypes: object(3)
memory usage: 604.8+ KB


In [4]:
# Revisemos si tiene valores faltantes
df_reviews.isna().sum()

user_id     0
user_url    0
reviews     0
dtype: int64

##### Columna review (desanidar)

In [5]:
df_reviews = df_reviews.explode("reviews").reset_index()
df_reviews = pd.concat([df_reviews.drop(columns="reviews"), df_reviews["reviews"].apply(pd.Series)], axis=1)
df_reviews

,index,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review,0
0,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,NaN
1,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,NaN
2,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,NaN
3,1,js41637,http://steamcommunity.com/id/js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,NaN
4,1,js41637,http://steamcommunity.com/id/js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,NaN
...,...,...,...,...,...,...,...,...,...,...,...
59328,25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,,Posted July 10.,,70,No ratings yet,True,a must have classic from steam definitely wort...,NaN
59329,25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,,Posted July 8.,,362890,No ratings yet,True,this game is a perfect remake of the original ...,NaN
59330,25798,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,1 person found this review funny,Posted July 3.,,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...,NaN
59331,25798,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,,Posted July 20.,,730,No ratings yet,True,:D,NaN


In [6]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59333 entries, 0 to 59332
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   index        59333 non-null  int64  
 1   user_id      59333 non-null  object 
 2   user_url     59333 non-null  object 
 3   funny        59305 non-null  object 
 4   posted       59305 non-null  object 
 5   last_edited  59305 non-null  object 
 6   item_id      59305 non-null  object 
 7   helpful      59305 non-null  object 
 8   recommend    59305 non-null  object 
 9   review       59305 non-null  object 
 10  0            0 non-null      float64
dtypes: float64(1), int64(1), object(9)
memory usage: 5.0+ MB


In [7]:
df_reviews.isna().sum()

index              0
user_id            0
user_url           0
funny             28
posted            28
last_edited       28
item_id           28
helpful           28
recommend         28
review            28
0              59333
dtype: int64

##### Eliminamos columnas no importantes para nuestro analisis y valores faltantes

In [8]:
df_reviews = df_reviews.drop(columns=[ "index","funny", "last_edited", "helpful", 0])
df_reviews = df_reviews.dropna(subset=['recommend', 'review', 'posted', 'item_id'])

In [9]:
# Verificamos si ya no hay valores faltantes 
df_reviews.isna().sum()

user_id      0
user_url     0
posted       0
item_id      0
recommend    0
review       0
dtype: int64

##### Veamos si hay datos duplicados 

In [10]:
# Veamos cuantos duplicados hay
df_reviews.duplicated().sum()

874

In [11]:
# Eliminamos los valores duplicados 
df_reviews.drop_duplicates()

,user_id,user_url,posted,item_id,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"Posted November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"Posted July 15, 2011.",22200,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"Posted April 21, 2011.",43110,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,http://steamcommunity.com/id/js41637,"Posted June 24, 2014.",251610,True,I know what you think when you see this title ...
4,js41637,http://steamcommunity.com/id/js41637,"Posted September 8, 2013.",227300,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...,...
59328,76561198312638244,http://steamcommunity.com/profiles/76561198312...,Posted July 10.,70,True,a must have classic from steam definitely wort...
59329,76561198312638244,http://steamcommunity.com/profiles/76561198312...,Posted July 8.,362890,True,this game is a perfect remake of the original ...
59330,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,Posted July 3.,273110,True,had so much fun plaing this and collecting res...
59331,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,Posted July 20.,730,True,:D


In [12]:
df_reviews.to_parquet("reviews.parquet")